In [159]:
import pandas as pd 
import numpy as np 
from matplotlib import pyplot as plt 
from IPython.display import display
import seaborn as sns

## Filter data

In [160]:
def filter_data(data_sent , max_flux= -12):
    data = data_sent.copy()
    max_flux = -12
    min_flux = 26
    data = data[data['flux_aper']<max_flux]

    data = data[data['significance']>2]
    data_class = data[['class']]

    data_sig = data['significance']
    data_id = data['src_id']
    data_name = data['src_n']
    obs_info_params = [ 'livetime','likelihood','pileup_flag','mstr_sat_src_flag','mstr_streak_src_flag'   ,'gti_obs' , 'flux_significance_b'  , 'flux_significance_m' , 'flux_significance_s' , 'flux_significance_h' , 'flux_significance_u'    ]
    data_val = data.drop(columns=obs_info_params)
    return data_val

def norm_data(data_sent):
    data = data_sent.copy()
    #data.replace()
    for d in data:
        max_val = np.amax(data[d])
        min_val =  np.amin(data[d])
        data[d] = (data[d]-min_val)/(max_val-min_val)
    return data
def std_data(data_sent):
    data = data_sent.copy()
    for d in data:
        mean =  np.mean(data[d])
        std = np.sqrt(np.var(data[d]))
        data[d] = (data[d]-mean)/std 
    return data
def do_nothing(data_sent):
    return data_sent

In [161]:
def extract_data(data_sent , impute_fn = '',reduce_fn = ' ' , rf_impute=False):
    data = data_sent.copy()
    data = data.sample(frac=1)
    data = filter_data(data)
    #display(data)
    data_id = data[[ 'class' ,'src_n' , 'src_id' ,'significance' , ]]
    data_id = data_id
    data_val = data.drop([ 'class' ,'src_n' , 'src_id' ,'significance' ,] , axis=1)
    data_val = reduce_fn(data_val)
    return data_val , data_id
    #if(rf_impute):
    #    data_val  , random_forest_imputer = impute_fn(data_val , data_id)
    #else:
    #    data_val = impute_fn(data_val)
    data_val = reduce_fn(data_val)
    data_val = data_val.reset_index(drop=True)
    data_reduced = pd.concat([data_id , data_val] , axis=1)
    if(rf_impute):
        return(data_reduced , random_forest_imputer)
    else:
        return data_reduced

## Load data

Here we will impute each class seperately ,
for testing we can pass given sample through each imputer and then combine the result

In [162]:
obj_class = 'PULSAR'

In [163]:

train = pd.read_csv('../processed_data/'+obj_class+'_.csv' , index_col='obs_id')
train = train.sample(frac=1)


In [164]:
train['class'].value_counts()

PULSAR    445
Name: class, dtype: int64

# RF imputer Missingpy

In [165]:
import sklearn.neighbors._base
from os import sys
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base

In [166]:
from missingpy import MissForest 

In [167]:
def rf_impute(d, i ):
    data = pd.concat([i , d] , axis=1)
    data = data.drop(columns=['src_n' , 'src_id' , 'significance' ,])
    rf_imputer = MissForest()
    #new_data = d.drop(columns= ['class'])
    new_data = rf_imputer.fit_transform(d)
    return new_data , rf_imputer

# Choose Features

In [168]:
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [169]:
from features import features as feat 
all_feat = list(feat['info'])+list(feat['flux']['photon'])+list(feat['flux']['energy'])+list(feat['variability'])+list(feat['hardness'])+list(feat['model_fit']['powerlaw'])+list(feat['model_fit']['bb'])+list(feat['model_fit']['brems'])+list(feat['info_pre_filter'])

In [170]:
feat_to_drop = list(feat['model_fit']['powerlaw']) + list(feat['model_fit']['bb']) + list(feat['model_fit']['brems']) 
#feat_to_drop = []
feat_used = [item for item in all_feat if item not in feat_to_drop]

In [171]:
train[feat_used]

,src_id,src_n,class,significance,photflux_aper_lolim_m,photflux_aper_lolim_h,photflux_aper_hilim_s,photflux_aper_hilim_b,photflux_aper_lolim_u,photflux_aper_hilim_m,...,flux_significance_m,gti_obs,flux_significance_b,pileup_flag,livetime,likelihood,mstr_streak_src_flag,flux_significance_u,mstr_sat_src_flag,flux_significance_h
obs_id,,,,,,,,,,,,,,,,,,,,,
PULSAR_OBS_147,PL0042,PSR J1124-591,PULSAR,144.29,-4.242908,-4.092266,-4.414539,-3.583359,-4.636200,-4.226360,...,52.54,2006-10-16T18:09:38,80.15,False,159127.1,62909.28000,False,4.00,False,56.94
PULSAR_OBS_326,PL0099,PSR J1833-103,PULSAR,121.78,-3.688882,-2.917574,-4.596365,-2.666150,-4.235973,-3.648784,...,21.75,2000-05-23T14:38:05,50.02,False,7736.0,20080.29000,False,2.32,False,44.78
PULSAR_OBS_60,PL0014,PSR J0525-6607,PULSAR,117.49,-3.872247,-4.042728,-3.832387,-3.286930,-4.642446,-3.850473,...,39.62,2009-09-19T00:27:40,57.65,False,26543.0,49419.92000,False,4.49,False,27.11
PULSAR_OBS_436,PL0130,PSR J2139+471,PULSAR,3.20,-6.707080,-6.689094,-6.222501,-5.707522,NaN,-6.221921,...,2.00,2013-03-03T09:09:15,3.20,False,24732.6,47.97266,False,0.00,False,1.84
PULSAR_OBS_419,PL0123,PSR J2022+3842,PULSAR,29.70,-5.158954,-4.392652,-6.121363,-4.211266,-6.623423,-5.092589,...,13.11,2005-08-01T04:28:46,29.70,False,53578.6,5063.46000,False,1.45,False,26.19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PULSAR_OBS_273,PL0081,PSR J1745-2900,PULSAR,107.91,-5.934794,-3.752763,-6.188559,-3.677574,-6.042968,-5.767258,...,5.27,2013-09-14T00:21:06,50.06,False,45410.1,21810.19000,False,1.35,False,49.34
PULSAR_OBS_276,PL0081,PSR J1745-2900,PULSAR,107.91,-6.638272,-4.880085,-7.100234,-4.733533,NaN,-6.275069,...,2.53,2014-04-28T03:04:50,12.18,False,44955.8,21810.19000,False,0.00,False,11.73
PULSAR_OBS_365,PL0099,PSR J1833-103,PULSAR,121.78,-4.081236,-3.258770,-5.061530,-3.092965,-5.481091,-4.027427,...,16.15,2000-07-05T06:08:23,37.12,False,7225.7,20080.29000,False,2.26,False,33.15


In [172]:
sp = [] 
for f in feat_used:
    #print(f)
    na = train[feat_used][f].isna().value_counts()
    try:
        sp.append([f , 1-na[0]/(na[0]+na[1])])
    except:
        sp.append([f , 0])
sp =  np.asarray(sp)
sparsity = pd.DataFrame(sp , columns=['feat' , 'sparsity']).sort_values(by='sparsity' , ascending=False)


In [173]:
sparsity

,feat,sparsity
33,flux_aper_lolim_u,0.5910112359550561
28,flux_aper_u,0.5640449438202247
8,photflux_aper_lolim_u,0.4831460674157303
11,photflux_aper_u,0.449438202247191
32,flux_aper_hilim_u,0.2741573033707865
...,...,...
66,likelihood,0
67,mstr_streak_src_flag,0
2,class,0
69,mstr_sat_src_flag,0


In [174]:
train_set = train[feat_used]

In [175]:
data_val , data_id   = extract_data(train_set ,  impute_fn= rf_impute , reduce_fn= do_nothing , rf_impute=True )

In [176]:
data_val

,photflux_aper_lolim_m,photflux_aper_lolim_h,photflux_aper_hilim_s,photflux_aper_hilim_b,photflux_aper_lolim_u,photflux_aper_hilim_m,photflux_aper_hilim_u,photflux_aper_u,photflux_aper_h,photflux_aper_hilim_h,...,var_index,hard_ms_hilim,hard_hm_lolim,hard_hs,hard_ms_lolim,hard_hs_hilim,hard_hm_hilim,hard_ms,hard_hs_lolim,hard_hm
obs_id,,,,,,,,,,,,,,,,,,,,,
PULSAR_OBS_3,-4.216454,-4.600499,-3.984640,-3.508218,-4.448428,-4.152612,-4.107238,-4.240786,-4.529443,-4.471855,...,1.20,-0.1405,-0.4478,-0.5309,-0.2405,-0.4716,-0.3079,-0.1911,-0.5890,-0.3773
PULSAR_OBS_242,NaN,-4.844360,-5.707301,-4.514421,NaN,-6.320663,NaN,NaN,-4.738499,-4.658368,...,6.00,-0.4241,0.9500,0.8969,-1.0000,0.9688,1.0000,-0.9994,0.7889,0.9994
PULSAR_OBS_133,-5.993534,-5.758454,-5.878769,-5.165198,NaN,-5.798057,-6.458171,NaN,-5.655215,-5.571703,...,0.25,0.2767,0.1230,0.3660,-0.0606,0.5066,0.4016,0.1074,0.2142,0.2673
PULSAR_OBS_211,-4.540608,-4.757459,-4.060381,-3.753994,-3.597739,-4.505706,-3.552532,-3.574466,-4.730254,-4.704433,...,0.00,-0.4578,-0.2680,-0.6396,-0.4941,-0.6190,-0.2005,-0.4760,-0.6577,-0.2336
PULSAR_OBS_435,-5.144541,-5.721018,-4.607479,-4.306097,-4.425737,-5.055468,-4.320300,-4.369877,-5.617263,-5.533429,...,0.00,-0.4428,-0.6127,-0.8095,-0.5328,-0.7714,-0.4466,-0.4884,-0.8463,-0.5309
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PULSAR_OBS_424,-5.671417,-5.572352,-6.063285,-4.930332,NaN,-5.475734,-4.765989,NaN,-5.457922,-5.371713,...,0.00,0.8576,-0.0294,0.7433,0.4641,0.8951,0.2717,0.6746,0.5603,0.1237
PULSAR_OBS_431,-6.937418,NaN,-5.951947,-5.651111,-5.807433,-6.072219,-4.951947,-5.187621,NaN,-6.464706,...,2.00,0.2567,-1.0000,-0.9994,-0.5890,-0.3204,-0.1918,-0.1961,-1.0000,-0.9994
PULSAR_OBS_220,-7.533726,-5.014349,-6.551912,-4.713544,NaN,-6.278437,-5.190642,NaN,-4.931072,-4.861382,...,0.00,1.0000,0.9126,0.9994,0.0606,1.0000,0.9900,0.9994,0.9550,0.9557


In [177]:
data_id 

,class,src_n,src_id,significance
obs_id,,,,
PULSAR_OBS_3,PULSAR,PSR J0100-7211,PL0003,65.51
PULSAR_OBS_242,PULSAR,PSR J1745-2900,PL0081,107.91
PULSAR_OBS_133,PULSAR,PSR J1048-5832,PL0033,7.43
PULSAR_OBS_211,PULSAR,PSR J1741-205,PL0079,98.07
PULSAR_OBS_435,PULSAR,PSR J2124-335,PL0129,17.47
...,...,...,...,...
PULSAR_OBS_424,PULSAR,PSR J2032+4127,PL0125,8.11
PULSAR_OBS_431,PULSAR,PSR J2051-0827,PL0128,5.69
PULSAR_OBS_220,PULSAR,PSR J1745-2900,PL0081,107.91


In [178]:
new_data , random_forest_imputer = rf_impute(data_val, data_id)
#train_data.index.name = 'index'
#display(train_data.describe())
#train_data.to_csv('../processed_data/train_norm_rf_impute')

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5


In [179]:
data_val

,photflux_aper_lolim_m,photflux_aper_lolim_h,photflux_aper_hilim_s,photflux_aper_hilim_b,photflux_aper_lolim_u,photflux_aper_hilim_m,photflux_aper_hilim_u,photflux_aper_u,photflux_aper_h,photflux_aper_hilim_h,...,var_index,hard_ms_hilim,hard_hm_lolim,hard_hs,hard_ms_lolim,hard_hs_hilim,hard_hm_hilim,hard_ms,hard_hs_lolim,hard_hm
obs_id,,,,,,,,,,,,,,,,,,,,,
PULSAR_OBS_3,-4.216454,-4.600499,-3.984640,-3.508218,-4.448428,-4.152612,-4.107238,-4.240786,-4.529443,-4.471855,...,1.20,-0.1405,-0.4478,-0.5309,-0.2405,-0.4716,-0.3079,-0.1911,-0.5890,-0.3773
PULSAR_OBS_242,NaN,-4.844360,-5.707301,-4.514421,NaN,-6.320663,NaN,NaN,-4.738499,-4.658368,...,6.00,-0.4241,0.9500,0.8969,-1.0000,0.9688,1.0000,-0.9994,0.7889,0.9994
PULSAR_OBS_133,-5.993534,-5.758454,-5.878769,-5.165198,NaN,-5.798057,-6.458171,NaN,-5.655215,-5.571703,...,0.25,0.2767,0.1230,0.3660,-0.0606,0.5066,0.4016,0.1074,0.2142,0.2673
PULSAR_OBS_211,-4.540608,-4.757459,-4.060381,-3.753994,-3.597739,-4.505706,-3.552532,-3.574466,-4.730254,-4.704433,...,0.00,-0.4578,-0.2680,-0.6396,-0.4941,-0.6190,-0.2005,-0.4760,-0.6577,-0.2336
PULSAR_OBS_435,-5.144541,-5.721018,-4.607479,-4.306097,-4.425737,-5.055468,-4.320300,-4.369877,-5.617263,-5.533429,...,0.00,-0.4428,-0.6127,-0.8095,-0.5328,-0.7714,-0.4466,-0.4884,-0.8463,-0.5309
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PULSAR_OBS_424,-5.671417,-5.572352,-6.063285,-4.930332,NaN,-5.475734,-4.765989,NaN,-5.457922,-5.371713,...,0.00,0.8576,-0.0294,0.7433,0.4641,0.8951,0.2717,0.6746,0.5603,0.1237
PULSAR_OBS_431,-6.937418,NaN,-5.951947,-5.651111,-5.807433,-6.072219,-4.951947,-5.187621,NaN,-6.464706,...,2.00,0.2567,-1.0000,-0.9994,-0.5890,-0.3204,-0.1918,-0.1961,-1.0000,-0.9994
PULSAR_OBS_220,-7.533726,-5.014349,-6.551912,-4.713544,NaN,-6.278437,-5.190642,NaN,-4.931072,-4.861382,...,0.00,1.0000,0.9126,0.9994,0.0606,1.0000,0.9900,0.9994,0.9550,0.9557


In [180]:
imp_data =  pd.DataFrame(new_data , columns = data_val.columns.to_list()  , index=data_val.index.to_list())
imp_data.index.name = 'obs_id'
imp_data

,photflux_aper_lolim_m,photflux_aper_lolim_h,photflux_aper_hilim_s,photflux_aper_hilim_b,photflux_aper_lolim_u,photflux_aper_hilim_m,photflux_aper_hilim_u,photflux_aper_u,photflux_aper_h,photflux_aper_hilim_h,...,var_index,hard_ms_hilim,hard_hm_lolim,hard_hs,hard_ms_lolim,hard_hs_hilim,hard_hm_hilim,hard_ms,hard_hs_lolim,hard_hm
obs_id,,,,,,,,,,,,,,,,,,,,,
PULSAR_OBS_3,-4.216454,-4.600499,-3.984640,-3.508218,-4.448428,-4.152612,-4.107238,-4.240786,-4.529443,-4.471855,...,1.20,-0.1405,-0.4478,-0.5309,-0.2405,-0.4716,-0.3079,-0.1911,-0.5890,-0.3773
PULSAR_OBS_242,-6.419807,-4.844360,-5.707301,-4.514421,-5.352059,-6.320663,-4.520899,-4.645552,-4.738499,-4.658368,...,6.00,-0.4241,0.9500,0.8969,-1.0000,0.9688,1.0000,-0.9994,0.7889,0.9994
PULSAR_OBS_133,-5.993534,-5.758454,-5.878769,-5.165198,-7.124184,-5.798057,-6.458171,-6.448039,-5.655215,-5.571703,...,0.25,0.2767,0.1230,0.3660,-0.0606,0.5066,0.4016,0.1074,0.2142,0.2673
PULSAR_OBS_211,-4.540608,-4.757459,-4.060381,-3.753994,-3.597739,-4.505706,-3.552532,-3.574466,-4.730254,-4.704433,...,0.00,-0.4578,-0.2680,-0.6396,-0.4941,-0.6190,-0.2005,-0.4760,-0.6577,-0.2336
PULSAR_OBS_435,-5.144541,-5.721018,-4.607479,-4.306097,-4.425737,-5.055468,-4.320300,-4.369877,-5.617263,-5.533429,...,0.00,-0.4428,-0.6127,-0.8095,-0.5328,-0.7714,-0.4466,-0.4884,-0.8463,-0.5309
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PULSAR_OBS_424,-5.671417,-5.572352,-6.063285,-4.930332,-5.457162,-5.475734,-4.765989,-5.067196,-5.457922,-5.371713,...,0.00,0.8576,-0.0294,0.7433,0.4641,0.8951,0.2717,0.6746,0.5603,0.1237
PULSAR_OBS_431,-6.937418,-7.534366,-5.951947,-5.651111,-5.807433,-6.072219,-4.951947,-5.187621,-6.691920,-6.464706,...,2.00,0.2567,-1.0000,-0.9994,-0.5890,-0.3204,-0.1918,-0.1961,-1.0000,-0.9994
PULSAR_OBS_220,-7.533726,-5.014349,-6.551912,-4.713544,-6.073981,-6.278437,-5.190642,-5.437291,-4.931072,-4.861382,...,0.00,1.0000,0.9126,0.9994,0.0606,1.0000,0.9900,0.9994,0.9550,0.9557


In [181]:
data_id

,class,src_n,src_id,significance
obs_id,,,,
PULSAR_OBS_3,PULSAR,PSR J0100-7211,PL0003,65.51
PULSAR_OBS_242,PULSAR,PSR J1745-2900,PL0081,107.91
PULSAR_OBS_133,PULSAR,PSR J1048-5832,PL0033,7.43
PULSAR_OBS_211,PULSAR,PSR J1741-205,PL0079,98.07
PULSAR_OBS_435,PULSAR,PSR J2124-335,PL0129,17.47
...,...,...,...,...
PULSAR_OBS_424,PULSAR,PSR J2032+4127,PL0125,8.11
PULSAR_OBS_431,PULSAR,PSR J2051-0827,PL0128,5.69
PULSAR_OBS_220,PULSAR,PSR J1745-2900,PL0081,107.91


In [182]:
processed_data_all = pd.concat([data_id , imp_data] , axis=1)
processed_data_all.to_csv('../processed_data/v3/'+obj_class+'_rf_impute.csv')

In [183]:
processed_data_all

,class,src_n,src_id,significance,photflux_aper_lolim_m,photflux_aper_lolim_h,photflux_aper_hilim_s,photflux_aper_hilim_b,photflux_aper_lolim_u,photflux_aper_hilim_m,...,var_index,hard_ms_hilim,hard_hm_lolim,hard_hs,hard_ms_lolim,hard_hs_hilim,hard_hm_hilim,hard_ms,hard_hs_lolim,hard_hm
obs_id,,,,,,,,,,,,,,,,,,,,,
PULSAR_OBS_3,PULSAR,PSR J0100-7211,PL0003,65.51,-4.216454,-4.600499,-3.984640,-3.508218,-4.448428,-4.152612,...,1.20,-0.1405,-0.4478,-0.5309,-0.2405,-0.4716,-0.3079,-0.1911,-0.5890,-0.3773
PULSAR_OBS_242,PULSAR,PSR J1745-2900,PL0081,107.91,-6.419807,-4.844360,-5.707301,-4.514421,-5.352059,-6.320663,...,6.00,-0.4241,0.9500,0.8969,-1.0000,0.9688,1.0000,-0.9994,0.7889,0.9994
PULSAR_OBS_133,PULSAR,PSR J1048-5832,PL0033,7.43,-5.993534,-5.758454,-5.878769,-5.165198,-7.124184,-5.798057,...,0.25,0.2767,0.1230,0.3660,-0.0606,0.5066,0.4016,0.1074,0.2142,0.2673
PULSAR_OBS_211,PULSAR,PSR J1741-205,PL0079,98.07,-4.540608,-4.757459,-4.060381,-3.753994,-3.597739,-4.505706,...,0.00,-0.4578,-0.2680,-0.6396,-0.4941,-0.6190,-0.2005,-0.4760,-0.6577,-0.2336
PULSAR_OBS_435,PULSAR,PSR J2124-335,PL0129,17.47,-5.144541,-5.721018,-4.607479,-4.306097,-4.425737,-5.055468,...,0.00,-0.4428,-0.6127,-0.8095,-0.5328,-0.7714,-0.4466,-0.4884,-0.8463,-0.5309
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PULSAR_OBS_424,PULSAR,PSR J2032+4127,PL0125,8.11,-5.671417,-5.572352,-6.063285,-4.930332,-5.457162,-5.475734,...,0.00,0.8576,-0.0294,0.7433,0.4641,0.8951,0.2717,0.6746,0.5603,0.1237
PULSAR_OBS_431,PULSAR,PSR J2051-0827,PL0128,5.69,-6.937418,-7.534366,-5.951947,-5.651111,-5.807433,-6.072219,...,2.00,0.2567,-1.0000,-0.9994,-0.5890,-0.3204,-0.1918,-0.1961,-1.0000,-0.9994
PULSAR_OBS_220,PULSAR,PSR J1745-2900,PL0081,107.91,-7.533726,-5.014349,-6.551912,-4.713544,-6.073981,-6.278437,...,0.00,1.0000,0.9126,0.9994,0.0606,1.0000,0.9900,0.9994,0.9550,0.9557
